In [ ]:
#import libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Ridge
from mapie.metrics import regression_coverage_score
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import explained_variance_score
from matplotlib.legend import _get_legend_handles_labels
from sklearn.model_selection import train_test_split

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"  # disable debugging logs from Tensorflow
import warnings
warnings.filterwarnings('ignore')
plt.rc('xtick',labelsize=19)
plt.rc('ytick',labelsize=19)
plt.rc('axes', labelsize=20, titlesize=16)
# To plot consistent and pretty figures
%matplotlib inline
import matplotlib as mpl
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
mpl.rcParams['font.family'] = 'times new roman'
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['figure.dpi'] = 120
mpl.rcParams['savefig.dpi'] = 120
def function(x):
    """One-dimensional x*sin(x) function."""
    return x * np.cos(x)

def data_het(
    funct, min_x, max_x, n_samples, noise
):
    """
    Generate 1D noisy data uniformly from the given function
    and standard deviation for the noise.
    Parameters:
        funct (function): The function used to generate the data.
        min_x (float): Minimum value for the x-axis range.
        max_x (float): Maximum value for the x-axis range.
        n_samples (int): Number of data samples to generate.
        noise (float): Standard deviation of the Gaussian noise to add to the data.
    
    Returns:
        X_train (numpy array): Training input data.
        y_train (numpy array): Noisy training output data.
        X_test (numpy array): Test input data.
        y_test (numpy array): Noisy test output data.
        y_mesh (numpy array): True output data without noise for test points.

    """
    np.random.seed(59)
    X_train = np.linspace(min_x, max_x, n_samples)
    np.random.shuffle(X_train)
    X_test = np.linspace(min_x, max_x, n_samples * 10)
    y_train = (
        funct(X_train) +
        (np.random.normal(0, noise, len(X_train)) * X_train)
    )
    y_test = (
        funct(X_test) +
        (np.random.normal(0, noise, len(X_test)) * X_test)
    )
    y_mesh = funct(X_test)
    return (
        X_train.reshape(-1, 1), y_train, X_test.reshape(-1, 1), y_test, y_mesh
    )


def jackknife_plus(X,Y,Xtest,Ytest,alpha):
    """
    Calculate prediction intervals using the jackknife-plus method.
    
    Parameters:
        X (numpy array): Training input data.
        Y (numpy array): Noisy training output data.
        Xtest (numpy array): Test input data.
        Ytest (numpy array): Noisy test output data.
        alpha (float): Confidence level (e.g., 0.95 for 95% confidence).
    
    Returns:
        res (list): List containing [alpha, coverage, width, PIs].
            - alpha (float): The specified confidence level.
            - coverage (float): The actual coverage probability of the prediction intervals.
            - width (float): The average width of the prediction intervals.
            - PIs (numpy array): Array of shape (ntest, 2) containing the prediction intervals for each test point.
    """
    n = X.shape[0]              
    ntest = Xtest.shape[0]         
    R = np.zeros(n)              
    y_pred=np.zeros((n,ntest))
    res=[]
    for i in np.arange(n):
        X_ = np.delete(X,i,0)
        Y_ = np.delete(Y,i)
        polyn_model.fit(X_, Y_)
        R[i] = np.abs(Y[i] - polyn_model.predict(X[i].reshape(1, -1)))
        y_pred[i]=polyn_model.predict(Xtest)
    PIs = np.zeros((ntest,2))    
    for itest in np.arange(ntest):
        q_lo = np.sort(y_pred[:,itest]-R)[::-1][(np.ceil((1-alpha)*(n+1))).astype(int)]
        q_hi = np.sort(y_pred[:,itest]+R)[(np.ceil((1-alpha)*(n+1))).astype(int)]
        PIs[itest] = np.array([q_lo,q_hi])
    coverage=round(regression_coverage_score(Ytest, PIs[:,0], PIs[:,1]),3)
    width=( PIs[:,1] - PIs[:,0]).mean().round(3)
    ypred = PIs.mean(axis=1)
    res.append([alpha,coverage,width,PIs])
    return res
def weighted_jackknife_plus(X,Y,Xtest,Ytest,alpha):
    """
    Calculate weighted prediction intervals using the weighted jackknife-plus method.
    
    Parameters:
        X (numpy array): Training input data.
        Y (numpy array): Noisy training output data.
        Xtest (numpy array): Test input data.
        Ytest (numpy array): Noisy test output data.
        alpha (float): Confidence level (e.g., 0.95 for 95% confidence).
    
    Returns:
        res (list): List containing [alpha, coverage, width, PIs].
            - alpha (float): The specified confidence level.
            - coverage (float): The actual coverage probability of the prediction intervals.
            - width (float): The average width of the prediction intervals.
            - PIs (numpy array): Array of shape (ntest, 2) containing the prediction intervals for each test point.
    """
    
    n = X.shape[0]              
    ntest = Xtest.shape[0]          
    R = np.zeros(n)              
    y_pred=np.zeros((n,ntest))
    d=np.zeros((n,ntest))
    w=np.zeros((n,ntest))
    res=[]
    for i in np.arange(n):
        X_ = np.delete(X,i,0)
        Y_ = np.delete(Y,i)
        polyn_model.fit(X_, Y_)
        R[i] = np.abs(Y[i] - polyn_model.predict(X[i].reshape(1, -1)))
        y_pred[i]=polyn_model.predict(Xtest)
        d[i]=1/(cdist(X[i].reshape(1,-1),Xtest,'euclidean')) 

    PIs = np.zeros((ntest,2))    
   
    for itest in np.arange(ntest):
        w[:,itest] = d[:,itest]/d[:,itest].sum()
        q_lo = np.sort(y_pred[:,itest]-(w[:,itest]*R*n))[::-1][(np.ceil((1-alpha)*(n+1))).astype(int)]
        q_hi = np.sort(y_pred[:,itest]+(w[:,itest]*R*n))[(np.ceil((1-alpha)*(n+1))).astype(int)]
        PIs[itest] = np.array([q_lo,q_hi])

    coverage=round(regression_coverage_score(Ytest, PIs[:,0], PIs[:,1]),3)
    width=( PIs[:,1] - PIs[:,0]).mean().round(3)
    ypred = PIs.mean(axis=1)
    res.append([alpha,coverage,width,PIs])
    return res


def plot(
    """
    Plot the training data, prediction intervals, and optionally the test data.
    
    Parameters:
        X_train (numpy array): Training input data.
        y_train (numpy array): Noisy training output data.
        X_test (numpy array): Test input data.
        y_test (numpy array): Noisy test output data.
        y_pred (numpy array): Mean predictions for the test points.
        y_pred_low (numpy array): Lower bounds of the prediction intervals.
        y_pred_up (numpy array): Upper bounds of the prediction intervals.
        ax (matplotlib axis, optional): The axis on which to plot the data. If None, a new figure and axis will be created.
        title (str, optional): Title for the plot.
    """
    X_train,
    y_train,
    X_test,
    y_test,
    y_pred,
    y_pred_low,
    y_pred_up,
    ax=None,
    title=None,
):
    ax.fill_between(X_test, y_pred_low, y_pred_up,color='lightgray', alpha=0.5, label="Prediction intervals")
    ax.scatter(X_train, y_train,color="red", s=10, alpha=0.3, label="Data point") 
#     ax.scatter(X_test, y_test,color="orange", s=1, alpha=0.3, label="Test data")
    if title is not None:
        ax.set_title(title,fontweight ='bold',fontsize=14)
